In [6]:
import pandas as pd

def read_n_preprocess(path):
        # path = 'output/by_Y_M.csv'
        df = pd.read_csv(path)
        df.drop('Unnamed: 0',axis = 1, inplace = True)
        # df.columns = ['Airport','Month','Average Wait Time (min)']
        return df

def make_airport_column(df):
    airport_column = []
    for airport in df.Airport:
        if airport not in airport_column:
            airport_column.append(airport)
        if len(airport_column) == len(set(df.Airport)):
            break
    num_months = int(df.shape[0] / len(airport_column))
    return airport_column, num_months

def make_wait_time_list(df,num_months):
    wait_time_list = []
    begin = 0
    for i in range(len(df)):
    # i + 1 so that the last set of data is included. Example if 47 is the last row, then i+1 = 48
        if ((i + 1) % num_months == 0) & (i != 0):
            to_append = list(df['Average_wait_time'][begin:i + 1])
            wait_time_list.append(to_append)
            begin = i + 1
#         print(i+1,"MOD ME")
    return wait_time_list

def make_airport_wait_time(airport_column, wait_time_list):
    airport_dict = {}
    i = 0
    for airport in airport_column:
        airport_dict[airport] = wait_time_list[i]
        i+=1
    return airport_dict

def make_airport_col_df(airport_column,airport_dict):
    airport_df = pd.DataFrame(columns=[airport for airport in airport_column])
    for airport in airport_column:
        airport_df[airport] = airport_dict[airport]
    return airport_df

def make_year_col_df(df,year):
    row_num = df.shape[0]
    some_df = pd.DataFrame({'Year':[year for i in range(row_num)]})
    return some_df

def make_month_col_df(num_months):
    month_df = pd.DataFrame({'Month':[i+1 for i in range(num_months)]})
    return month_df

def process_by_year(df):
    yearly_df_list = []
    for year in set(df.Year):
        df_year = df[df.Year == year]
        airport_col, num_months = make_airport_column(df_year)
        wait_time_list = make_wait_time_list(df_year,num_months)
        airport_dict = make_airport_wait_time(airport_col,wait_time_list)

        airport_df = make_airport_col_df(airport_col,airport_dict)
        year_df = make_year_col_df(df_year,year)
        month_df = make_month_col_df(num_months)
        time_df = concat_df_cols(year_df,month_df)
        new_df = concat_df_cols(time_df,airport_df)
        yearly_df_list.append(new_df)
    return yearly_df_list

def concat_df_cols(df1,df2):
    concatinated_df = pd.concat([df1,df2],axis = 1)
    return concatinated_df

def concat_df_rows(df1,df2):
    concatinated_df = pd.concat([df1,df2],axis = 0)
    return concatinated_df

def remove_na(df_list):
    clean_df_list = []
    for df in df_list:
        df_clean = df.dropna()
        clean_df_list.append(df_clean)
    return clean_df_list

def final_concat(clean_df_list):
    all_years_data = pd.concat(clean_df_list)
    return all_years_data


def save(new_df):
    new_df.to_csv("output/Month by Month Report.csv",index = False)

In [7]:
df = read_n_preprocess('output/by_Y_M.csv')
dirty_yearly_list = process_by_year(df)
clean_yearly_list = remove_na(dirty_yearly_list)
final_df = final_concat(clean_yearly_list)
save(final_df)

In [8]:
final_df

,Year,Month,BOS,EWR,JFK,PHL
0,2016,1.0,16.48,14.22,21.29,16.15
1,2016,2.0,13.19,11.51,18.02,10.22
2,2016,3.0,16.01,13.36,21.53,14.06
3,2016,4.0,14.98,14.42,22.49,13.38
4,2016,5.0,16.06,14.76,19.99,13.97
5,2016,6.0,17.61,15.59,19.13,12.47
6,2016,7.0,19.02,17.83,21.34,15.21
7,2016,8.0,21.96,19.09,23.05,16.06
8,2016,9.0,20.17,15.73,21.37,13.72
9,2016,10.0,17.23,15.62,20.35,11.99


In [ ]:
yearly_df_list[1]